In [2]:
from pyspark import SparkConf, SparkContext
import pandas as pd
import numpy as np
from math import sqrt

In [3]:
sc = SparkContext.getOrCreate()

In [3]:
rating_data = sc.textFile("ml-100k/u.data")
movie_data = sc.textFile("ml-100k/u.item")

In [4]:
# sim_movie = sc.textFile("movie-sims/").persist()
sim_movie = sc.pickleFile("movie-sims-obj/").persist()

In [5]:
sim_movie.takeSample(False, 5)

[((367, 1380), (1.0, 1)),
 ((396, 915), (0.9379987913699099, 5)),
 ((748, 1251), (0.96, 2)),
 ((657, 1015), (0.9801960588196069, 3)),
 ((276, 1357), (0.9486832980505138, 2))]

In [38]:
sim_movie_local = sim_movie.collect()
sim_dict = {}
for sm in sim_movie_local:
    key = sm[0]
    value = sm[1]
    
    sim_dict[key] = value

sim_dict

{(224, 893): (0.8395701571521512, 2),
 (224, 894): (0.9899494936611665, 2),
 (224, 895): (0.9381026756366438, 6),
 (224, 896): (0.9626875454708417, 8),
 (224, 897): (1.0, 1),
 (224, 898): (0.9203900230262253, 3),
 (224, 899): (0.729396808625797, 3),
 (224, 900): (0.9585587314447452, 8),
 (224, 901): (0.8466416999071569, 3),
 (224, 902): (0.9609331989577097, 7),
 (224, 903): (0.9704949588309457, 4),
 (224, 904): (0.8116794499134277, 3),
 (224, 905): (0.9999999999999998, 2),
 (224, 906): (0.9033950326094178, 6),
 (224, 907): (1.0, 1),
 (224, 908): (1.0, 1),
 (224, 909): (0.9946917938265513, 2),
 (224, 910): (0.9429903335828895, 2),
 (224, 911): (0.9899494936611665, 2),
 (224, 912): (0.9429903335828895, 2),
 (224, 913): (0.7761140001162655, 2),
 (224, 914): (0.9429903335828895, 2),
 (224, 915): (0.9995120760870788, 2),
 (224, 916): (0.9838699100999074, 2),
 (224, 917): (1.0, 1),
 (224, 918): (0.9984603532054125, 2),
 (224, 919): (0.9748057177323376, 15),
 (224, 920): (1.0, 1),
 (224, 921)

In [6]:
def _extract_user_rating(line):
    data = line.split('\t')
    return (int(data[0]), [(int(data[1]), float(data[2]))])

In [7]:
user_lists = rating_data.map(_extract_user_rating).reduceByKey(lambda v1,v2: v1+v2)
user_lists.takeSample(False, 5)

[(318,
  [(474, 4.0),
   (768, 2.0),
   (356, 4.0),
   (257, 5.0),
   (631, 4.0),
   (659, 4.0),
   (50, 2.0),
   (476, 4.0),
   (64, 4.0),
   (648, 5.0),
   (508, 4.0),
   (204, 5.0),
   (610, 5.0),
   (660, 3.0),
   (357, 4.0),
   (47, 2.0),
   (286, 3.0),
   (72, 4.0),
   (795, 2.0),
   (1014, 2.0),
   (524, 3.0),
   (248, 3.0),
   (197, 5.0),
   (1032, 3.0),
   (194, 5.0),
   (315, 5.0),
   (503, 4.0),
   (239, 4.0),
   (49, 3.0),
   (384, 3.0),
   (934, 4.0),
   (481, 4.0),
   (1023, 2.0),
   (396, 1.0),
   (750, 4.0),
   (182, 4.0),
   (305, 2.0),
   (401, 3.0),
   (66, 4.0),
   (509, 5.0),
   (655, 4.0),
   (127, 5.0),
   (88, 4.0),
   (501, 4.0),
   (12, 4.0),
   (105, 1.0),
   (419, 5.0),
   (160, 3.0),
   (575, 2.0),
   (137, 4.0),
   (158, 5.0),
   (162, 5.0),
   (485, 5.0),
   (237, 5.0),
   (63, 3.0),
   (435, 5.0),
   (186, 5.0),
   (414, 4.0),
   (15, 5.0),
   (796, 3.0),
   (815, 3.0),
   (1521, 3.0),
   (140, 4.0),
   (211, 5.0),
   (326, 4.0),
   (312, 4.0),
   (657, 

In [8]:
user_id = 816
predicted_movie = 660

In [9]:
_, user_ratings = user_lists.filter(lambda line: line[0]==user_id).collect()[0]

In [37]:
numerator = 0
denominator = 0

for movie_id, rating in user_ratings:
    if(predicted_movie < movie_id):
        m1 = predicted_movie
        m2 = movie_id
    else:
        m2 = predicted_movie
        m1 = movie_id
    
    sim_data = sim_movie.filter(lambda line: line[0][0]==m1 and line[0][1]==m2).collect()
    sim_score, number_of_record = sim_data[0][1]
    
    numerator += sim_score*rating
    denominator += sim_score
    
#     print(sim_score)
#     numerator = rating * 

predicted_rating = numerator/denominator
print(predicted_rating)

3.855887801610851


In [42]:
numerator = 0
denominator = 0

for movie_id, rating in user_ratings:
    if(predicted_movie < movie_id):
        m1 = predicted_movie
        m2 = movie_id
    else:
        m2 = predicted_movie
        m1 = movie_id
    
    sim_score, number_of_record = sim_dict[(m1, m2)]
    
    numerator += sim_score*rating
    denominator += sim_score
    
#     print(sim_score)
#     numerator = rating * 

predicted_rating = numerator/denominator
print(predicted_rating)

3.855887801610851


# Movie_data

In [4]:
movie_data = sc.textFile("ml-100k/u.item")
movie_data.takeSample(False, 5)

['1656|Little City (1998)|20-Feb-1998||http://us.imdb.com/M/title-exact?Little+City+(1998)|0|0|0|0|0|1|0|0|0|0|0|0|0|0|1|0|0|0|0',
 '838|In the Line of Duty 2 (1987)|30-Aug-1996||http://us.imdb.com/M/title-exact?In%20the%20Line%20of%20Duty%202%20(1987)|0|1|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0',
 '1522|Trial by Jury (1994)|01-Jan-1994||http://us.imdb.com/M/title-exact?Trial%20by%20Jury%20(1994)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0',
 '922|Dead Man (1995)|10-May-1996||http://us.imdb.com/M/title-exact?Dead%20Man%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1',
 "1079|Joe's Apartment (1996)|26-Jul-1996||http://us.imdb.com/M/title-exact?Joe's%20Apartment%20(1996)|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0|0|0|0"]

In [5]:
def _extract_movie_data(line):
    data = line.split('|')
    return (int(data[0]), data[1])

movie_local = movie_data.map(_extract_movie_data).collect()
movie_local

[(1, 'Toy Story (1995)'),
 (2, 'GoldenEye (1995)'),
 (3, 'Four Rooms (1995)'),
 (4, 'Get Shorty (1995)'),
 (5, 'Copycat (1995)'),
 (6, 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)'),
 (7, 'Twelve Monkeys (1995)'),
 (8, 'Babe (1995)'),
 (9, 'Dead Man Walking (1995)'),
 (10, 'Richard III (1995)'),
 (11, 'Seven (Se7en) (1995)'),
 (12, 'Usual Suspects, The (1995)'),
 (13, 'Mighty Aphrodite (1995)'),
 (14, 'Postino, Il (1994)'),
 (15, "Mr. Holland's Opus (1995)"),
 (16, 'French Twist (Gazon maudit) (1995)'),
 (17, 'From Dusk Till Dawn (1996)'),
 (18, 'White Balloon, The (1995)'),
 (19, "Antonia's Line (1995)"),
 (20, 'Angels and Insects (1995)'),
 (21, 'Muppet Treasure Island (1996)'),
 (22, 'Braveheart (1995)'),
 (23, 'Taxi Driver (1976)'),
 (24, 'Rumble in the Bronx (1995)'),
 (25, 'Birdcage, The (1996)'),
 (26, 'Brothers McMullen, The (1995)'),
 (27, 'Bad Boys (1995)'),
 (28, 'Apollo 13 (1995)'),
 (29, 'Batman Forever (1995)'),
 (30, 'Belle de jour (1967)'),
 (31, 'Crimson Tide 

In [6]:
movie_dict = dict(movie_local)

# scale

In [32]:
user_ratings = sorted(user_ratings, key=lambda x: x[0])
user_ratings

[(243, 4.0),
 (258, 3.0),
 (259, 2.0),
 (260, 3.0),
 (264, 4.0),
 (271, 4.0),
 (288, 4.0),
 (294, 5.0),
 (300, 4.0),
 (309, 5.0),
 (313, 5.0),
 (322, 4.0),
 (323, 4.0),
 (326, 4.0),
 (328, 4.0),
 (331, 5.0),
 (332, 4.0),
 (342, 4.0),
 (343, 4.0),
 (349, 4.0),
 (355, 2.0),
 (678, 4.0),
 (687, 2.0),
 (690, 4.0),
 (1025, 4.0)]

In [33]:
watched_list = [movie for movie, rating in user_ratings]
rating_list = [rating for movie, rating in user_ratings]

In [34]:
if(predicted_movie < movie_id):
    m1 = predicted_movie
    m2 = movie_id
else:
    m2 = predicted_movie
    m1 = movie_id

sim_data = sim_movie.filter(lambda line: line[0][0]==m1 and line[0][1]==m2).collect()

In [35]:
predicted_movie = 660
watched_list = []
for movie, rating in user_ratings:
    if (movie<predicted_movie):
        watched_list += [(movie, predicted_movie)]
    else:
        watched_list += [(predicted_movie, movie)]
watched_list

[(243, 660),
 (258, 660),
 (259, 660),
 (260, 660),
 (264, 660),
 (271, 660),
 (288, 660),
 (294, 660),
 (300, 660),
 (309, 660),
 (313, 660),
 (322, 660),
 (323, 660),
 (326, 660),
 (328, 660),
 (331, 660),
 (332, 660),
 (342, 660),
 (343, 660),
 (349, 660),
 (355, 660),
 (660, 678),
 (660, 687),
 (660, 690),
 (660, 1025)]

In [36]:
sim_data = sim_movie.filter(lambda line: line[0] in watched_list)
sim_data.collect()

[((243, 660), (0.8954850827371652, 21)),
 ((258, 660), (0.9636351420920835, 86)),
 ((259, 660), (0.8707358123825878, 34)),
 ((260, 660), (0.8657181027857811, 21)),
 ((264, 660), (0.9430419201928972, 14)),
 ((271, 660), (0.9000681867284961, 27)),
 ((288, 660), (0.9259131484300386, 73)),
 ((294, 660), (0.9100191293354817, 82)),
 ((300, 660), (0.9197491700948728, 57)),
 ((309, 660), (0.9782838736655738, 4)),
 ((313, 660), (0.9529778052200882, 69)),
 ((322, 660), (0.9428554577104868, 32)),
 ((323, 660), (0.9181863939785798, 35)),
 ((326, 660), (0.9434469816028902, 20)),
 ((328, 660), (0.9438457587928833, 44)),
 ((331, 660), (0.9487844046582881, 9)),
 ((332, 660), (0.9396116091200583, 22)),
 ((342, 660), (0.9935591240195244, 4)),
 ((343, 660), (0.8980251179515535, 16)),
 ((349, 660), (0.9515226836973145, 3)),
 ((355, 660), (0.9260923597695477, 2)),
 ((660, 678), (0.9236156140907448, 36)),
 ((660, 687), (0.8499999999999999, 14)),
 ((660, 690), (0.9221514421792558, 21)),
 ((660, 1025), (0.900